### Load libraries

In [1]:
# Library for data visualisation
import pandas as pd
import numpy as np
from pathlib import Path
import pathlib
import os
from PIL import Image
import numpy as np
import json
from cv2 import cv2
import torch
from torchvision.ops import box_convert
from typing import List, Dict
from skimage.io import imread
import pickle

#for model
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead, RegionProposalNetwork
import torch

import torchvision.transforms as T
from cv2 import cv2
import os
import json
import time
import pickle

import pathlib
from torch.utils.data import DataLoader
import sys
sys.path.append('../')

from tqdm import tqdm

#for model
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead, RegionProposalNetwork
import torch


## Create a test train folder

In [2]:
import os
import random
import numpy as np
import shutil
from cv2 import cv2
import json
import pathlib
import sys
sys.path.append('../')

#Set path to root dir (wrist folder)
root_dir = '../literature/Other/supervisely/'

# Path to annotation dir
ann_dir = os.listdir(root_dir + 'wrist/ann')

# Path to image dir
img_dir = os.listdir(root_dir + 'wrist/img')



print('Number of annotations: {}'.format(len(ann_dir)))
print('Number of images: {}'.format(len(img_dir)))




Number of annotations: 20274
Number of images: 20274


In [5]:
#os.chdir('../')

print("The working directory of this project is:", os.getcwd())

The working directory of this project is: c:\Users\spiro\OneDrive\Υπολογιστής\Thesis\Project\master_thesis_dtu\notebooks


## Save images as RPG

In [63]:
os.mkdir("../literature/Other/supervisely/wrist/rpg_images")

In [64]:
from tqdm import tqdm
root_dir = pathlib.Path('../literature/Other/supervisely/wrist/img/')


for idx in tqdm(range(len(img_dir))):
    img_path = os.path.join(root_dir,img_dir[idx])
    ann_path = os.path.join('../literature/Other/supervisely/wrist/ann/',ann_dir[idx])


    img_gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    # Convert the grayscale image to RGB and duplicate the grayscale channel
    img = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2RGB) 
    cv2.imwrite("../literature/Other/supervisely/wrist/rpg_images/" +img_dir[idx] , img)


100%|██████████| 20274/20274 [22:09<00:00, 15.25it/s] 


## Create the pickles


In [65]:
from tqdm import tqdm
def data_fix(img_list,ann_list):

    root_dir = pathlib.Path('../literature/Other/supervisely')

    for idx in tqdm(range(len(img_list))):
        img_path = os.path.join(img_list[idx])
        ann_path = os.path.join(ann_list[idx])
        
        #target = []
        d = {}

        box = []
        boxes = []
        labels = []
        area = []

        classes = {'fracture' : 1, 'text' : 2}


        name = pathlib.PurePath(ann_path).name.split('.')[0]

        

        with open(ann_path) as json_file:
            
            #Load the JSON file
            data = json.load(json_file)


        for object_dict in data['objects']:
    
            # Check if object contains any fractures

            if object_dict['classTitle'] == "text" or object_dict['classTitle'] == "fracture":

                # Get points and convert them to int for display purposes
                top_left_point, bottom_right_point = object_dict['points']['exterior']
                top_left_point = list(map(int,top_left_point))
                bottom_right_point = list(map(int, bottom_right_point))
                box = (top_left_point+bottom_right_point)
                area.append((box[3] - box[1]) * (box[2] - box[0]))
                label = classes[object_dict['classTitle']]
                labels.append(label)
                boxes.append(box)

                # Check if the list is empty
                if len(box) == 0:
                    print("The list is empty!")
                

        boxes = torch.FloatTensor(boxes)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        area = torch.as_tensor(area, dtype=torch.int64)

        image_id = torch.tensor([idx])
                
        
        d["boxes"] = boxes
        d["labels"] = labels
        d["image_id"] = image_id
        d["area"] = area
        

        target = d

        img = img_path

        ready_data = {'image' : img,'target' : target}


        path = os.path.join(root_dir,'wrist','train_pickles', name + '.pickle')

        with open(path, 'wb') as handle:
            pickle.dump(ready_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [68]:
#Set path to root dir (wrist folder)
root_dir = pathlib.Path('../literature/Other/supervisely')

# Path to annotation dir
ann_dir = pathlib.Path(root_dir, 'wrist/ann')

# Path to image dir
img_dir = pathlib.Path(root_dir, 'wrist/rpg_images')

# Lists
list_files = lambda start_dir: [str(item) for item in start_dir.iterdir()]
ann_list = list_files(ann_dir)
img_list = list_files(img_dir)

print('Number of annotations: {}'.format(len(ann_list)))
print('Number of images: {}'.format(len(img_list)))

Number of annotations: 20274
Number of images: 20274


In [69]:
data_fix(img_list,ann_list)

100%|██████████| 20274/20274 [01:57<00:00, 173.04it/s]


## Create train-test folders 


In [70]:
os.mkdir("../literature/Other/supervisely/wrist/validation_pickles")
os.mkdir("../literature/Other/supervisely/wrist/test_pickles")

In [71]:
#MOVE THE TEST PICKLES TO ANOTHER FOLDER
#Set path to root dir (wrist folder)
root_dir = pathlib.Path('../literature/Other/supervisely')
pickles = pathlib.Path(root_dir, 'wrist/train_pickles')
pickle_list = os.listdir(pickles)


random.seed(1234)
test_images = random.sample(pickle_list, 3000)


for pickle in test_images:
    shutil.move('../literature/Other/supervisely/wrist/train_pickles/'+pickle, '../literature/Other/supervisely/wrist/test_pickles/'+pickle)

In [72]:
#MOVE SOME IMAGES FOR VALIDATION
root_dir = pathlib.Path('../literature/Other/supervisely')
pickles = pathlib.Path(root_dir, 'wrist/train_pickles')
pickle_list = os.listdir(pickles)
validation_images = random.sample(pickle_list, 1000)

for pickle in validation_images:
    shutil.move('../literature/Other/supervisely/wrist/train_pickles/'+pickle, '../literature/Other/supervisely/wrist/validation_pickles/'+pickle)

In [73]:
print(len(os.listdir('../literature/Other/supervisely/wrist/train_pickles')))
print(len(os.listdir('../literature/Other/supervisely/wrist/validation_pickles')))
print(len(os.listdir('../literature/Other/supervisely/wrist/test_pickles')))

16274
1000
3000


## Fix the DETR dataset


In [3]:
#Set path to root dir (wrist folder)
root_dir = pathlib.Path('../literature/Other/supervisely')

# Path to annotation dir
ann_dir = pathlib.Path(root_dir, 'wrist/ann')

# Path to image dir
img_dir = pathlib.Path(root_dir, 'wrist/rpg_images')

# Lists
list_files = lambda start_dir: [str(item) for item in start_dir.iterdir()]
ann_list = list_files(ann_dir)
img_list = list_files(img_dir)
print('Number of annotations: {}'.format(len(ann_list)))
print('Number of annotations: {}'.format(len(img_list)))

Number of annotations: 20274
Number of annotations: 20274


In [5]:
ann_dir

WindowsPath('../literature/Other/supervisely/wrist/ann')

In [3]:
import random


# combine the two lists into a list of tuples
image_annotation_pairs = list(zip(img_list, ann_list))

# shuffle the list of image and annotation pairs randomly
random.shuffle(image_annotation_pairs)

# calculate the length of the shuffled image and annotation pairs list
n = len(image_annotation_pairs)

# calculate the split points for the three lists
train_split = int(0.75 * n)  # 75% for training
test_split = int(0.15 * n)  # 15% for testing
val_split = n - train_split - test_split  # remaining 5% for validation

# split the shuffled image and annotation pairs list into three separate lists
train_pairs = image_annotation_pairs[:train_split]
test_pairs = image_annotation_pairs[train_split:train_split+test_split]
val_pairs = image_annotation_pairs[train_split+test_split:]

# unzip the pairs into separate lists of images and annotations
train_images, train_annotations = zip(*train_pairs)
test_images, test_annotations = zip(*test_pairs)
val_images, val_annotations = zip(*val_pairs)

# print the lengths of the three lists
print("Number of image/annotation pairs for training:", len(train_pairs))
print("Number of image/annotation pairs for testing:", len(test_pairs))
print("Number of image/annotation pairs for validation:", len(val_pairs))


Number of image/annotation pairs for training: 15205
Number of image/annotation pairs for testing: 3041
Number of image/annotation pairs for validation: 2028


In [4]:
from src.models.utils import create_COCO_from_dataset

create_COCO_from_dataset(train_images, train_annotations,"train")
create_COCO_from_dataset(test_images, test_annotations,"test")
create_COCO_from_dataset(val_images, val_annotations,"val")

100%|██████████| 2028/2028 [00:35<00:00, 57.20it/s]


{'info': {'description': 'My dataset in COCO format',
  'url': 'nan',
  'version': '1.0',
  'year': 2023,
  'contributor': 'Spyros Vlachospyros',
  'date_created': '2023-03-02'},
 'licenses': [{'id': 1,
   'name': 'CC-BY-4.0',
   'url': 'https://creativecommons.org/licenses/by/4.0/'}],
 'categories': [{'id': 0, 'name': 'text'}, {'id': 1, 'name': 'fracture'}],
 'images': [{'id': 1,
   'file_name': '../literature/Other/supervisely/wrist/rpg_images/2049_0572510983_03_WRI-L1_M015.png',
   'width': 628,
   'height': 1132},
  {'id': 2,
   'file_name': '../literature/Other/supervisely/wrist/rpg_images/0227_0911857525_03_WRI-L2_M009.png',
   'width': 573,
   'height': 730},
  {'id': 3,
   'file_name': '../literature/Other/supervisely/wrist/rpg_images/3574_0959344574_02_WRI-R2_M012.png',
   'width': 605,
   'height': 1044},
  {'id': 4,
   'file_name': '../literature/Other/supervisely/wrist/rpg_images/2199_0663140199_01_WRI-R1_F007.png',
   'width': 507,
   'height': 808},
  {'id': 5,
   'file_n

## Fix the validation set of CNN to a COCO fromat


In [2]:
%cd ..

c:\Users\spiro\OneDrive\Υπολογιστής\Thesis\Project\master_thesis_dtu


In [3]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from src.data.my_rpg_dataset import XRayDataSet_coco
from src.data.my_rpg_dataset import collate_fn


In [7]:
%cd notebooks

c:\Users\spiro\OneDrive\Υπολογιστής\Thesis\Project\master_thesis_dtu\notebooks


In [4]:
# Create a dictionary to hold the converted annotations
coco_annotations = {
    'images': [],
    'annotations': [],
    'categories': [{'id': 1, 'name': 'fracture'}, {'id': 2, 'name': 'text'}]
}

#load test data
test_dataset = XRayDataSet_coco(pathlib.Path('../literature/Other/supervisely/wrist/test_pickles'))
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4,collate_fn=collate_fn)


# Convert your annotations to COCO format
for i, (img_path, targets) in enumerate(test_dataset):
        
        
    file_name = img_path
        

    # Load the image using cv2
    image = cv2.imread(img_path)

    # Get the height and width of the image
    height, width, channels = image.shape

    coco_annotations['images'].append({
        'id': int(targets['image_id'].cpu().numpy()[0]),
        'file_name': file_name.replace("\\","/").replace('../', ''),
        'height': height,
        'width': width
    })
    for j in range(len(targets['boxes'])):
        #turn boxex from xmin,ymin,xmax,ymax format to coco format x,y,w,h
        box = targets['boxes'][j]
        x, y, w, h = box[0], box[1], box[2] - box[0], box[3] - box[1] 
        area = (box[3] - box[1]) * (box[2] - box[0])
        coco_annotations['annotations'].append({
            'id': len(coco_annotations['annotations']) + 1,
            'image_id': int(targets['image_id'].cpu().numpy()[0]),
            'category_id': targets['labels'][j].item(),
            'bbox': [x.item(), y.item(), w.item(), h.item()],
            'area': area.item(),
            'iscrowd': 0  # Assuming all instances are not crowd
        })

# Create a COCO object for your annotations
coco_gt = COCO()
coco_gt.dataset = coco_annotations
coco_gt.createIndex()

creating index...
index created!


In [11]:
# Save the COCO object to a JSON file
with open('test_coco_gt.json', 'w') as f:
    json.dump(coco_gt.dataset, f)

In [12]:
# Load the COCO object from a JSON file
with open('coco_gt.json', 'r') as f:
    coco_gt_data = json.load(f)
coco_gt = COCO()
coco_gt.dataset = coco_gt_data
coco_gt.createIndex()

creating index...
index created!
